# 2. Продвинутые операции

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Test") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

## 2.1 Оконные функции

Оконные функции (Window Functions) в PySpark позволяют выполнять вычисления над группами строк, сохраняя при этом индивидуальность каждой строки.

Основные компоненты оконных функций:
1. Оконная спецификация (WindowSpec) - определяет, какие строки будут включены в рамки окна для каждой строки
2. Оконная функция - функция, которая применяется к данным в рамках окна

**Ранжирование** (rank(), dense_rank(), row_number(), percent_rank())  
**Агрегатные функции** (sum(), avg(), max(), min())  
**Смещение** (lag(), lead())  
**Аналитические функции** (first(), last(), cume_dist(), ntile())  

In [2]:
# Задача: Найти разницу между текущей и предыдущей покупкой для каждого пользователя.

# Создаём DataFrame с покупками
sales = spark.createDataFrame([
    (1, "2023-01-10", 100),
    (1, "2023-01-15", 200),
    (2, "2023-01-12", 50),
    (1, "2023-01-20", 300)
], ["user_id", "date", "amount"])

# Определяем окно
from pyspark.sql.window import Window
window = Window.partitionBy("user_id").orderBy("date")

# Добавляем разницу с предыдущей покупкой
from pyspark.sql.functions import lag, col
sales_with_diff = sales.withColumn("prev_amount", lag("amount").over(window)).withColumn("diff", col("amount") - col("prev_amount"))
sales_with_diff.show()

+-------+----------+------+-----------+----+
|user_id|      date|amount|prev_amount|diff|
+-------+----------+------+-----------+----+
|      1|2023-01-10|   100|       NULL|NULL|
|      1|2023-01-15|   200|        100| 100|
|      1|2023-01-20|   300|        200| 100|
|      2|2023-01-12|    50|       NULL|NULL|
+-------+----------+------+-----------+----+



In [3]:
# Топ-3 товара по категориям

# Данные о товарах
products = spark.createDataFrame([
    (1, "Laptop", "Electronics", 999),
    (2, "Phone", "Electronics", 699),
    (3, "Desk", "Furniture", 200),
    (4, "Chair", "Furniture", 150)
], ["product_id", "name", "category", "price"])

# Окно для ранжирования
window = Window.partitionBy("category").orderBy(col("price").desc())

# Топ-3 в каждой категории
from pyspark.sql.functions import dense_rank
top_products = products.withColumn("rank", dense_rank().over(window)).filter(col("rank") <= 3)
top_products.show()

+----------+------+-----------+-----+----+
|product_id|  name|   category|price|rank|
+----------+------+-----------+-----+----+
|         1|Laptop|Electronics|  999|   1|
|         2| Phone|Electronics|  699|   2|
|         3|  Desk|  Furniture|  200|   1|
|         4| Chair|  Furniture|  150|   2|
+----------+------+-----------+-----+----+



## 2.2 Работа с датами и строками

Обработка дат и времени:
- Парсинг строк в даты: to_date(), to_timestamp()
- Извлечение компонентов: year(), month(), dayofweek()
- Арифметика: datediff(), date_add(), months_between()
- Форматирование: date_format()

In [4]:
# Анализ продаж по времени

spark = SparkSession.builder.appName("Data").getOrCreate()

# Создаём DataFrame с датами
sales_data = [
    (1, "2023-01-15", 150.0),
    (2, "15-02-2023", 200.0),  # Нестандартный формат
    (3, "2023/03/20", 99.99)
]
df = spark.createDataFrame(sales_data, ["order_id", "order_date", "amount"])

# Приводим даты к единому формату (yyyy-MM-dd)
df = df.withColumn("parsed_date", 
      to_date(col("order_date"), "yyyy-MM-dd")) \
      .withColumn("parsed_date", 
      coalesce(col("parsed_date"), 
               to_date(col("order_date"), "dd-MM-yyyy"),
               to_date(col("order_date"), "yyyy/MM/dd")))

# Извлекаем год, месяц и день недели
df = df.withColumn("year", year("parsed_date")) \
       .withColumn("month", month("parsed_date")) \
       .withColumn("day_of_week", dayofweek("parsed_date"))

df.show()

+--------+----------+------+-----------+----+-----+-----------+
|order_id|order_date|amount|parsed_date|year|month|day_of_week|
+--------+----------+------+-----------+----+-----+-----------+
|       1|2023-01-15| 150.0| 2023-01-15|2023|    1|          1|
|       2|15-02-2023| 200.0| 2023-02-15|2023|    2|          4|
|       3|2023/03/20| 99.99| 2023-03-20|2023|    3|          2|
+--------+----------+------+-----------+----+-----+-----------+



In [5]:
# Расчёт дней между заказами
from pyspark.sql.window import Window

window = Window.partitionBy("year").orderBy("parsed_date")
df = df.withColumn("days_since_last_order", 
      datediff(col("parsed_date"), lag("parsed_date").over(window)))

df.show()

+--------+----------+------+-----------+----+-----+-----------+---------------------+
|order_id|order_date|amount|parsed_date|year|month|day_of_week|days_since_last_order|
+--------+----------+------+-----------+----+-----+-----------+---------------------+
|       1|2023-01-15| 150.0| 2023-01-15|2023|    1|          1|                 NULL|
|       2|15-02-2023| 200.0| 2023-02-15|2023|    2|          4|                   31|
|       3|2023/03/20| 99.99| 2023-03-20|2023|    3|          2|                   33|
+--------+----------+------+-----------+----+-----+-----------+---------------------+



Работа со строками:
- Базовые операции: concat(), substring(), trim()
- Регулярные выражения: regexp_extract(), regexp_replace()
- Проверки: startswith(), endswith(), contains()

In [6]:
# Очистка и анализ текста

# Извлечение домена из email
users = spark.createDataFrame([
    (1, "alice@example.com"),
    (2, "bob@gmail.com")
], ["user_id", "email"])

users = users.withColumn("domain", 
      regexp_extract(col("email"), "@(.+)$", 1))

users.show()

+-------+-----------------+-----------+
|user_id|            email|     domain|
+-------+-----------------+-----------+
|      1|alice@example.com|example.com|
|      2|    bob@gmail.com|  gmail.com|
+-------+-----------------+-----------+



In [7]:
# Замена цензурой нецензурных слов
comments = spark.createDataFrame([
    (1, "This is bad!"),
    (2, "Worst product ever")
], ["comment_id", "text"])

bad_words = ["bad", "worst"]
pattern = "|".join(bad_words)
comments = comments.withColumn("clean_text", 
      regexp_replace(col("text"), pattern, "***"))

comments.show()

+----------+------------------+------------------+
|comment_id|              text|        clean_text|
+----------+------------------+------------------+
|         1|      This is bad!|      This is ***!|
|         2|Worst product ever|Worst product ever|
+----------+------------------+------------------+



In [8]:
# Загрузка CSV с заголовком и выводом схемы
df_Titanic = spark.read.csv("data/train.csv", header=True, inferSchema=True)

# Разделяем "Surname, Title Name" и извлекаем фамилию
df_Titanic = df_Titanic.withColumn("surname", 
      split(col("Name"), ",")[0])

# Длина имени
df_Titanic = df_Titanic.withColumn("name_length", 
      length(col("Name")))

df_Titanic.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+---------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|  surname|name_length|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+---------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|   Braund|         23|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|  Cumings|         51|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|Heikkinen|         22|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S| Futrelle|         44|
|          5|       0|     3|Allen, Mr. W

## 2.3 Оптимизация (партиционирование, кэширование)

Партиционирование — разделение данных на части (партиции) для параллельной обработки. Большие файлы (например, 100+ ГБ CSV/Parquet). Частые фильтры по определённым колонкам (например, по дате).

Пример
```
# Запись данных с партиционированием по колонке 'Pclass' (Titanic dataset)
df.write.partitionBy("Pclass").parquet("data/titanic_partitioned")

# Чтение только партиции Pclass=1 (ускоряет загрузку в 3+ раза)
df_pclass1 = spark.read.parquet("data/titanic_partitioned/Pclass=1")
```

```
# Репартиционирование в 200 партиций (для равномерного распределения)
df_repartitioned = df.repartition(200)

# Партиционирование по колонке + указание числа партиций
df_repartitioned = df.repartition(10, "Pclass")
```

Кэширование сохраняет данные в памяти или на диске для повторного использования. Если DataFrame используется более 2-х раз. Перед многократными итерациями (например, в ML-циклах).  
Уровни хранения:
- `MEMORY_ONLY` — только в памяти (быстро, но риск OOM).
- `MEMORY_AND_DISK` — сначала память, потом диск (надежнее).
- `DISK_ONLY` — только диск (медленно, но безопасно).

Пример
```
from pyspark.storagelevel import StorageLevel

# Кэширование в памяти (аналог .cache())
df.cache()  

# Явное указание уровня
df.persist(StorageLevel.MEMORY_AND_DISK)

# Проверка статуса
df.is_cached  # -> True

# Освобождение памяти
df.unpersist()
```

```
# Плохо: DataFrame читается 2 раза
df.filter(df.Age > 30).count()
df.filter(df.Age > 30).show()

# Хорошо: Кэшируем перед повторным использованием
filtered_df = df.filter(df.Age > 30).persist()
filtered_df.count()
filtered_df.show()
```

### Настройка памяти и параметров Spark

```
spark = SparkSession.builder \
    .appName("Optimization") \
    .config("spark.executor.memory", "4g") \  # Память на executor
    .config("spark.driver.memory", "2g") \    # Память на driver
    .config("spark.sql.shuffle.partitions", "200") \  # Число партиций после shuffle
    .master("local[*]") \
    .getOrCreate()
```

Shuffle — перераспределение данных между узлами (например, при groupBy или join).  
Как уменьшить нагрузку:
- Увеличьте spark.sql.shuffle.partitions (по умолчанию 200).
- Используйте broadcast для маленьких таблиц:
```
from pyspark.sql.functions import broadcast
df1.join(broadcast(df2), "key")
```

Мониторинг и отладка. Откройте в браузере: `http://localhost:4040` (для локального режима).

| Ошибка | Причина | Решение |
| -- | -- | -- |
| OutOfMemoryError | Нехватка памяти | Увеличьте spark.executor.memory |
| Data skew | Неравномерное распределение данных | Используйте repartition или salting |
| Slow JOIN | Большие таблицы без оптимизации | Примените broadcast для малых таблиц |

## 2.4 UDF (пользовательские функции)

User-Defined Functions — это пользовательская функция, которую можно применять к колонкам DataFrame.

Когда встроенных функций Spark недостаточно. Для применения сложной Python-логики (например, NLP, машинное обучение).

UDF работают медленнее встроенных функций (из-за накладных расходов на сериализацию). Для ускорения используют Pandas UDF (векторизованные функции).

In [9]:
# Функция для категоризации возраста.
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName("UDFs").getOrCreate()

# Данные
data = [("Alice", 25), ("Bob", 60), ("Charlie", 15)]
df = spark.createDataFrame(data, ["name", "age"])

# Шаг 1: Создаём Python-функцию
def age_category(age):
    if age < 18:
        return "child"
    elif age >= 60:
        return "senior"
    else:
        return "adult"

# Шаг 2: Регистрируем UDF (указываем тип возвращаемого значения)
age_category_udf = udf(age_category, StringType())

# Шаг 3: Применяем к DataFrame
df = df.withColumn("age_category", age_category_udf(df["age"]))
df.show()

+-------+---+------------+
|   name|age|age_category|
+-------+---+------------+
|  Alice| 25|       adult|
|    Bob| 60|      senior|
|Charlie| 15|       child|
+-------+---+------------+



Векторизованные Pandas UDF (быстрые). Pandas UDF работают в 10-100 раз быстрее обычных UDF, т.к. обрабатывают данные партиями.

In [10]:
# Расчёт квадрата числа через Pandas UDF.

from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import IntegerType
import pandas as pd

# Шаг 1: Создаём векторную функцию
@pandas_udf(IntegerType())
def square_udf(age: pd.Series) -> pd.Series:
    return age ** 2

# Шаг 2: Применяем
df = df.withColumn("age_squared", square_udf(df["age"]))
df.show()

+-------+---+------------+-----------+
|   name|age|age_category|age_squared|
+-------+---+------------+-----------+
|  Alice| 25|       adult|        625|
|    Bob| 60|      senior|       3600|
|Charlie| 15|       child|        225|
+-------+---+------------+-----------+



Кэширование результатов. Если UDF вызывается многократно, закэшируйте DataFrame: `df.cache()`

In [11]:
# UDF, которая преобразует колонку Name в формат ФАМИЛИЯ, 
# Имя (например, "Braund, Mr. Owen Harris" → "BRAUND, Owen").
def format_name(name):
    surname = name.split(",")[0].upper()
    first_name = name.split(",")[1].split(".")[1].strip()
    return f"{surname}, {first_name}"

format_name_udf = udf(format_name, StringType())
df_Titanic = df_Titanic.withColumn("formatted_name", format_name_udf(df_Titanic["Name"]))

# Pandas UDF для расчёта длины строки в колонке Name.
@pandas_udf(IntegerType())
def name_length_udf(names: pd.Series) -> pd.Series:
    return names.str.len()

df_Titanic = df_Titanic.withColumn("name_length", name_length_udf(df_Titanic["Name"]))

# 3. Тест скорости
import time

start = time.time()
df.withColumn("name_length", name_length_udf(df["Name"])).count()
print(f"Pandas UDF: {time.time() - start} sec")

start = time.time()
df.withColumn("formatted_name", format_name_udf(df["Name"])).count()
print(f"Regular UDF: {time.time() - start} sec")

Pandas UDF: 8.301344633102417 sec
Regular UDF: 8.37052869796753 sec


## 2.5 Чтение/запись в разных форматах

| Формат | Описание | Плюсы | Минусы |
| - | - | - | - |
| CSV | Текстовый формат с разделителями | Простота, человекочитаемость | Нет схемы, медленный |
| JSON | Полуструктурированный формат | Поддержка вложенных данных | Медленный, большой объём |
| Parquet | Бинарный колоночный формат | Быстрый, сжатый, с поддержкой схемы | Нечитаем для человека |
| Avro | Бинарный формат с схемой | Эффективен для сериализации | Требует схемы |

### Чтение данных

Чтение данных CSV

In [12]:
# Использовать первую строку как заголовок
# Автоматически определить типы данных
# Указать разделитель
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .csv("data/train.csv")

df_csv.show(2)

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
only showing top 2 rows



Для больших файлов лучше явно указать схему (ускоряет загрузку)
```
schema = StructType([
    StructField("PassengerId", IntegerType()),
    StructField("Survived", IntegerType()),
    # ... остальные поля
])

df_csv = spark.read.schema(schema).csv("data/train.csv")
```

Чтение данных JSON
```
df_json = spark.read \
    .option("multiLine", "true") \      # Для многострочных JSON
    .json("data/amazon_sales.json")

df_json.printSchema()
```
Если JSON содержит вложенные структуры, используйте select + explode:
```
from pyspark.sql.functions import explode

df_exploded = df_json.select("user_id", explode("orders").alias("order"))
df_exploded.select("order.*").show()
```

Parquet (рекомендуемый формат для Big Data)
```
df_parquet = spark.read.parquet("data/titanic.parquet")
```
Почему Parquet?  
- Сжатие данных (экономия места).
- Колоночная организация (быстрые агрегации).
- Поддержка схемы.

Чтение данных Avro (требует пакета spark-avro)
```
df_avro = spark.read \
    .format("avro") \
    .load("data/titanic.avro")
```

Установка:
```
spark-submit --packages org.apache.spark:spark-avro_2.12:3.5.0 your_script.py
```

### Запись данных

CSV 
```
df_csv.write \
    .option("header", "true") \
    .option("delimiter", "|") \    # Изменяем разделитель
    .mode("overwrite") \           # Перезаписать, если файл существует
    .csv("output/titanic_csv")
```
Parquet 
```
df.write \
    .partitionBy("Pclass") \       # Партиционирование по колонке
    .mode("overwrite") \
    .parquet("output/titanic_parquet")
```
JSON
```
df.write \
    .mode("overwrite") \
    .json("output/titanic_json")
```
Avro
```
df.write \
    .format("avro") \
    .mode("overwrite") \
    .save("output/titanic.avro")
```

### Конвертация между форматами

Конвертация CSV в Parquet (для ускорения последующих запросов).
```
df_csv = spark.read.csv("data/titanic.csv", header=True, inferSchema=True)
df_csv.write.parquet("data/titanic_parquet")
```